The goal of this part is to fit the model to a sparse matrix with some games' FPS are missing. 
The key part is to compute the MSE with some element of F representing the fps data is missing.

## Building a model to predict FPS
First, we build our modle as:
\begin{align} 
F^{i}_{mn}=g^{i}P_{mn}+\alpha_{mn}
\end{align}
where $i$ is the label for games, $mn$ are the label for gpu and cpu respectively, and $\alpha$ contains other information that is game independent.

Next, I will use the current data to testify this model.

The number of parameters in this model is $i+2*m*n$.
Because we find 'i' games fps benchmark, the number of data point will be $i*m*n$.
In the case where $i=24,m=28,n=14$,the number of parameters are $808$ and the number of data points is $9408$.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
## find2d returna a 2d array representing fps among 
## different cpu, gpu combinations given a game
def find2d(game_name):
    game_fps=pd.read_csv(game_name+'.csv')
    game_fps=np.array(game_fps)
    game_fps=[i[1:] for i in game_fps]
    game_fps=np.array(game_fps).astype(np.float32)
    return game_fps

def load_fps_data():
    game_names=pd.read_csv('games_fps_hyperlinks.csv')['game_name']
    fps_all=np.array([find2d(i) for i in game_names])
    print('The training data contains {} games tested,\n'.format(fps_all.shape[0]), 
          '{} gpu types,'.format(fps_all.shape[1]), 
          '{} cpu types'.format(fps_all.shape[2]))
    return fps_all

In [ ]:
## Here I will use tensor flow to do it.
class model():
    def __init__(self):
        self.g=tf.Variable(tf.random.truncated_normal(shape=(24,)))
        self.P=tf.Variable(tf.random.truncated_normal(shape=(28,14)))
        self.alpha=tf.Variable(tf.random.truncated_normal(shape=(28,14)))
        self.trainable_variables=[self.P,self.alpha,self.g]
    def __call__(self):
        F_predict=tf.concat([tf.expand_dims(self.g[i]*self.P,0) for i in range(24)],0)\
                    +tf.tile(tf.expand_dims(self.alpha,0),[24,1,1])
        return F_predict
    def load_variables(self,parameters,g):
        self.P=tf.constant(parameters[0])
        self.alpha=tf.constant(parameters[1])
        if g:
            self.g=tf.constant(parameters[2])
        else:
            self.g=tf.Variable(tf.random.truncated_normal(shape=(1,)))
            self.trainable_variables=
        

In [ ]:
def compute_loss(F_predict, F):
    count=0
    SE=0
    for i in range(F.shape[0]):
        for j in range(F.shape[1]):
            for k in range(F.shape[2]):
                if F[i,j,k]:
                    SE=SE+tf.math.square(F_predict[i,j,k]-F[i,j,k])
                    count+=1   
    return SE/count

In [ ]:


def train_one_step(model,F):
    with tf.GradientTape() as tape:
        F_predict = model()
        loss = compute_loss(F_predict,F) 
        # compute gradient
        grads = tape.gradient(loss, model.trainable_variables)
        # update to weights
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
def build(model,epochs,F):   
    for epoch in range(epochs):           
        train_one_step(model,F)  
        if epoch%10==0:
            F_predict=model()
            print('for epoch {}, MSE is {}'.format(epoch,compute_loss(F_predict,F)))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

In [ ]:
testmodel=model()

In [ ]:
epochs=300
build(testmodel,epochs,fps_all)

In [ ]:
def save_model(model,path):
    stored_variables=np.array([i.numpy() for i in model.trainable_variables])
    np.save(path, stored_variables,allow_pickle=True, fix_imports=True)

In [ ]:
def load_predict(model,path,g=False):
    parameters=np.load('savedmodel.npy',allow_pickle=True)
    if g:
        model.load_variables(parameters)

initializer=[(fps,gpu_number,cpu_number),......]

In [ ]:
def predict(model,initializer):
    if not initializer:
        raise ValueError('initializer should not be empty')
    else:
        predict=[]
        observe=[]
        for test in initializer:
            observe.append(test[0])
            predict.append(model()[0][test[1]][test[2]])

In [ ]:
x=tf.Variable(tf.random.truncated_normal(shape=(1,28,14)))

In [ ]:
x[0][12][1]